In [1]:
# Install in google colab servers xmltodict
!pip install xmltodict


In [2]:
import pandas as pd
import xmltodict
import json
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry


# Let's define some functions to easily retrieve data from NERC


In [32]:
# Some function to deal with NERC
def get_nvs_variable_info(id=None,
                          variable=None,
                          vocabulary=None,
                          nvs_url="http://vocab.nerc.ac.uk/collection/",
                          version="current",
                          format_output="?_profile=skos&_mediatype=application/ld+json"
                          ):
    """
    Method to parse the json format from the NERC NVS servers
    """
    if id:
        url = id
    else:
        # Define the base of the URL
        url = nvs_url + '/' + vocabulary + '/' + version

        # Add the optional variable
        if variable:
            url = url + '/' + variable

    # Get the response from the NERC servers
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)

    response = session.get(url + '/' + format_output)
    return response.json()

def split_nerc_id(id_url):
    """
    Small method to parse the NERC url for each variables to extract information
    """
    # Split the order ids to extract name and vocab
    id_list = id_url.split('/')
    val = ['http', 'empty', 'nerc_url', 'type', 'vocabulary', 'version', 'variable', 'unknown']
    return dict(zip(val, id_list))

def regroup(input):
    output = ','.join(list({a for a in input if type(a) in [str]}))
    return output

def generate_json(df,ouput_path='dfo_vocabulary_list.json'):
    """
    This function suppose that the input data frame has a multiindex of
     2 level with vocabulary first and variable
     """
    # There is only one duplicated value both in IML and BIO associated with CNTR variable not sure why
    output_dict = {}
    for org, df_org in df.groupby(level=0):
        output_dict.update({org:{}})
        for var, df_var in df_org.groupby(level=1):
            if len(df_var)==1:
                output_dict[org].update({var:df_var.to_dict(orient='records')[0]})
            elif len(df_var)>1:
                output_dict[org].update({var:df_var.to_dict(orient='records')})
    with open(ouput_path,'w') as file_out:
        json.dump(output_dict,file_out,indent=2)
    return output_dict


# Combine MEDS and BIO Variables 
Merge the two and retrieve more information

In [4]:
# MEDS and BIO data
meds_list = 'https://raw.githubusercontent.com/cioos-siooc/cioos-siooc_data_transform/dev/projects/odf_transform/vocabulary/meds_pcodes_20191212_mods_utf8.csv'
bio_list = 'https://raw.githubusercontent.com/cioos-siooc/cioos-siooc_data_transform/dev/projects/odf_transform/vocabulary/bio_gf3_p01_mapping_2.7.2.xlsx'
mli_list = "https://raw.githubusercontent.com/cioos-siooc/cioos-siooc_data_transform/dev/projects/odf_transform/vocabulary/MLI_QO_dict_md.csv"

In [5]:
# Format MEDS and BIO data
# Get MEDS List
df_meds = pd.read_csv(meds_list) \
        .dropna(how='all', axis='index')\
        .rename({'CODE': 'GF3_CODE'}, axis='columns')
        

# Get BIO 
df_bio = pd.read_excel(bio_list) \
    .rename({'GF3(BIO) code': 'GF3_CODE',
             'standard_name':'CF_CODE'}, axis='columns') \
    .dropna(how='all', axis='index')
# Specify that the BIO file is related to BIO
df_bio['OWNER'] = 'BIO'  # Add bio as own to bio list



## Add more from MLI


In [6]:
# Load MLI list
df_mli = pd.read_csv(mli_list)

# Some columns have an organization.code. Let's split them
#for col in ['arbr_gen_dict_md', 'cle_dict_md', 'arbr_simpl_dic_md', 'abr_feuil_dict_md']:
for col in ['arbr_gen_dict_md', 'cle_dict_md', 'arbr_simpl_dic_md', 'abr_feuil_dict_md']:
    df_mli = df_mli.merge(df_mli[col].str.split('.').apply(pd.Series).rename({0:'org-'+col,1:'code-'+col},axis='columns'),
                 how='outer',left_index=True,right_index=True)

# def_dict_md has the units, let's isolate them
df_mli['units'] = df_mli['def_dict_md'].str.extract(r'(\([^\(\)]*\))$')[0].str.replace('^\(|\)$','')
df_mli['units'] = df_mli['units'].fillna(df_mli['def_dict_md'].str.extract(r'Unit.+:(.*)$')[0]
                                         .str.replace('.$|\'|\[|\]','')
                                         .str.replace('^\s+|\s+$',''))
# Fix some units and try to translate them
df_mli['units'] = df_mli['units'].str.replace('[mM]olair','mole')\
    .str.replace('[Dd]egr[eé]{1,2}s','degrees')\
    .str.replace('³','**3')\
    .str.replace('pourcentage','%')

# Filter the columns
mli_dict_conversion = {"def_dict_md":"definition","acr_dict_md":"GF3_CODE","acr_sn_md":"OWNER"}
df_mli = df_mli.rename(mli_dict_conversion,axis='columns')[["OWNER","definition","GF3_CODE","units"]].replace({'MPO':'MEDS'})


In [7]:
# Combine the three sources together
mergeBy = ['GF3_CODE','OWNER']
#df_bio = df_bio.rename({"BIO:OWNER":"OWNER","BIO:GF3_CODE":"GF3_CODE"},axis='columns')
df_group2 = df_meds.set_index(mergeBy).add_prefix('MEDS:').merge(df_bio.set_index(mergeBy).add_prefix('BIO:'),left_index=True,right_index=True,how='outer')
df_group2 = df_group2.merge(df_mli.set_index(mergeBy).add_prefix('IML:'),left_index=True,right_index=True,how='outer')
df_group2 = df_group2.replace({pd.NA:pd.NA,'nan':pd.NA,'unknown':pd.NA,\
                               'inconnu':pd.NA,'<NA>':pd.NA})

In [8]:
# Let's exclude NODC and IOS from the rest since they don't use BODC or not relevant for CIOOS
df_dfo = df_group2.query('OWNER=="NODC"').assign(Vocabulary = 'NODC').reset_index()
df_dfo = df_dfo.append(df_group2.query('OWNER=="IOS"').assign(Vocabulary = 'IOS').reset_index())
df_dfo = df_dfo.append(df_group2.query('OWNER=="QO"').assign(Vocabulary = 'QO').reset_index())
df_dfo = df_dfo.append(df_group2.query('OWNER=="CSIRO"').assign(Vocabulary = 'CSIRO').reset_index())

def count_unique(a):
    return len(set(a)-set([pd.NA,None,'']))

# Let's review MEDS,BIO and IML all together
#  We'll review each of the rows and find differences while ignoring the empty values
gf3_query = 'OWNER=="BIO" | OWNER=="MEDS" | OWNER=="IML" | OWNER=="GF3"'
n_version = df_group2.query(gf3_query).groupby(level=0).agg(count_unique)\
    .drop('MEDS:WMO_CODE_TABLE_ID',axis='columns')
#n_version[(n_version>1).any(axis='columns')]

# Move all variables that aren't repeated
not_repeated_var = df_group2.applymap(str).query(gf3_query).reset_index()\
    .groupby('GF3_CODE').agg(['first',','.join])\
    .loc[(n_version<2).any(axis='columns')]\
    .swaplevel(-2,-1,axis='columns')


df_dfo = df_dfo.append(not_repeated_var['first'].drop('OWNER',axis='columns')\
    .assign(OWNER = not_repeated_var['join']['OWNER'])\
    .assign(Vocabulary='GF3').reset_index())

# Add all the ones not matching at the end
not_matching = df_group2.applymap(str).query(gf3_query)\
    .loc[n_version[(n_version>1).any(axis='columns')].index]\
    .reset_index()

df_dfo = df_dfo.append(not_matching.assign(Vocabulary=not_matching['OWNER']))

# Retrieve the CF Standard Name Table and compare it to the DFO ones

In [9]:
# Get CF variable list and alias
response = requests.get('https://cfconventions.org/Data/cf-standard-names/77/src/cf-standard-name-table.xml', stream=True)
response.raw.decode_content = True
cf_dict = xmltodict.parse(response.text)

# Convert to dataframes
cf_alias_dict = {entry['@id']:entry['entry_id'] for entry in cf_dict['standard_name_table']['alias']}
df_cf = pd.DataFrame(cf_dict['standard_name_table']['entry'])
df_cf['version_number'] = cf_dict['standard_name_table']['version_number']
df_cf['last_modified'] = cf_dict['standard_name_table']['last_modified']
df_cf['institution'] = cf_dict['standard_name_table']['institution']
df_cf['contact'] = cf_dict['standard_name_table']['contact']
print(str(len(df_cf))+' standard_name available')

4460 standard_name available


In [10]:
# Review CF names provided by MEDS and BIO
# Replace CF alias by standard name if an alias is used instead
df_dfo[['MEDS:CF_CODE','BIO:CF_CODE']]= df_dfo[['MEDS:CF_CODE','BIO:CF_CODE']].replace(cf_alias_dict)

# Transfer CF names from MEDS and BIO to a general CF_CODE
# MEDS CF available in CF77
replace_rule = df_dfo['MEDS:CF_CODE'].isin(df_cf['@id'])
df_dfo.loc[replace_rule,'CF_CODE']  = df_dfo.loc[replace_rule,'MEDS:CF_CODE']
# BIO CF available in CF77           
replace_rule = df_dfo['BIO:CF_CODE'].isin(df_cf['@id']) & df_dfo['CF_CODE'].isna()
df_dfo.loc[replace_rule,'CF_CODE'] = df_dfo.loc[replace_rule,'BIO:CF_CODE']

# Rename CF_Code to CF recommended standard_name
df_dfo = df_dfo.rename({'CF_CODE':'standard_name'},axis='columns') 

df_dfo

,GF3_CODE,OWNER,MEDS:STATUS,MEDS:CATEGORY_MEANING_E,MEDS:CATEGORY_MEANING_F,MEDS:UNITS_DESCRIPTOR_E,MEDS:UNITS_DESCRIPTOR_F,MEDS:CF_CODE,MEDS:NETCDF_LONG_NAME,MEDS:WMO_CODE_TABLE_ID,MEDS:CONVENTION,BIO:units,BIO:P06 urn,BIO:P06 name,BIO:P01 urn,BIO:P01 name,BIO:CF_CODE,IML:definition,IML:units,Vocabulary,standard_name
0,AC2$,NODC,A,General purpose,Tout usage,text,texte,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NODC,NaN
1,ACC$,NODC,A,General purpose,Tout usage,text,texte,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NODC,NaN
2,ACCS,NODC,A,General purpose,Tout usage,text,texte,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NODC,NaN
3,ADDP,NODC,A,General purpose,Tout usage,text,texte,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NODC,NaN
4,ADID,NODC,A,General purpose,Tout usage,text,texte,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NODC,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,TRAN,IML,A,Biological Oceanography,Océanographie biologique,percent,pourcentage,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,IML,NaN
16,TRAN,MEDS,A,Physical Oceanography,Océanographie physique,percent,pourcentage,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Transmissivité. Unités: '%',%,MEDS,NaN
17,WDIR,BIO,A,Meteorology,Météorologie,degree(angle),degré(angle),<NA>,<NA>,<NA>,True,degrees,UAAA,degrees,<NA>,<NA>,wind_to_direction,<NA>,<NA>,BIO,wind_to_direction
18,WDIR,IML,A,Meteorology,Météorologie,degree(angle),degré(angle),<NA>,<NA>,<NA>,ToTrue,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,IML,NaN


# Retrieve the NERC ID related to each standard name in P07

In [11]:
# Retrieve NERC P07 matching ID
p07_dict = get_nvs_variable_info(vocabulary='P07')
df_P07 = pd.DataFrame.from_dict(p07_dict)
df_P07['standard_name'] = df_P07['http://www.w3.org/2004/02/skos/core#prefLabel'].apply(pd.Series)[0].apply(pd.Series)['@value']
df_dfo = df_dfo.merge(df_P07[['@id','standard_name']], on='standard_name',how='left').rename({'@id':'SDN:P07::id'},axis='columns')


# Retrieve NERC ID for each P01 available

In [12]:
# Retrieve NERC P01 ID from BIO:P01 urn
p01_dict = get_nvs_variable_info(vocabulary='P01')
df_P01 = pd.DataFrame.from_dict(p01_dict)
df_P01['SDN:P01::urn'] = df_P01['@id'].str.extract('(/[A-Z0-9a-z]*/$)')[0].str.replace('/','')
df_P01['SDN:P01::prefLabel'] = df_P01['http://www.w3.org/2004/02/skos/core#prefLabel'].apply(pd.Series)[0].apply(pd.Series)['@value']

df_dfo = df_dfo.merge(df_P01[['@id','SDN:P01::urn','SDN:P01::prefLabel']],
                      left_on='BIO:P01 urn', right_on='SDN:P01::urn',
                      how='left').rename({'@id':'SDN:P01::id'},axis='columns')

# Retrieve SeaDataNet Format for NetCDF attributes
df_dfo['sdn_parameter_urn'] = 'SDN:P01::'+df_dfo['SDN:P01::urn']
df_dfo['sdn_parameter_name'] = df_dfo['SDN:P01::prefLabel']

# Retrieve NERC ID for each P06 available

In [13]:
# Retrieve 
p06_dict = get_nvs_variable_info(vocabulary='P06')
df_P06 = pd.DataFrame.from_dict(p06_dict)
df_P06['SDN:P06::urn'] = df_P06['@id'].str.extract('/(\w*)/$')[0]
df_P06['SDN:P06::prefLabel'] = df_P06['http://www.w3.org/2004/02/skos/core#prefLabel'].apply(pd.Series)[0].apply(pd.Series)['@value']

df_dfo = df_dfo.merge(df_P06[['@id','SDN:P06::urn','SDN:P06::prefLabel']],
                      left_on='BIO:P06 urn', right_on='SDN:P06::urn',
                      how='left').rename({'@id':'SDN:P06::id'},axis='columns')


df_dfo['sdn_uom_urn'] = 'SDN:P06::'+df_dfo['SDN:P06::urn']
df_dfo['sdn_uom_name'] = df_dfo['SDN:P06::prefLabel']

# Generate Outputs to be used by Reviewers and data-transform


In [39]:
# Regroup the different information 
#homogenize empty values
empty_values = {'nan':pd.NA, pd.NA:pd.NA,'':pd.NA,'inconnu':pd.NA,\
                         'unknown':pd.NA,'<NA>':pd.NA}
df_dfo = df_dfo.replace(empty_values)
df_dfo = df_dfo.rename({'GF3_CODE':'name','OWNER':'Users'},axis='columns')

units_replacement = {'%':'percent','met(re|er)s+':'metre', "degrees+":'degree','\(angle\)':""}

df_dfo['expected_units'] = df_dfo[['MEDS:UNITS_DESCRIPTOR_E','BIO:units','IML:units']]\
    .replace(units_replacement,regex=True)\
    .apply(lambda row: ','.join(set(row.dropna().values)), axis=1)
df_dfo['definition'] = df_dfo[['IML:definition','SDN:P01::prefLabel']]\
    .apply(lambda row: ','.join(row.dropna().drop_duplicates().values), axis=1)
df_dfo['convention'] = df_dfo[['MEDS:CONVENTION']]\
    .apply(lambda row: ','.join(row.dropna().drop_duplicates().values), axis=1)


In [40]:
# If we want to remove the different MEDS, BIO and IML related columns
var_to_keep = ['Vocabulary',	'name',	'OWNER', 'expected_units',	'definition',	'convention' ,
               'standard_name', 'SDN:P07::id',
               'SDN:P01::id',	'SDN:P01::urn',	'SDN:P01::prefLabel',
               'sdn_parameter_urn',	'sdn_parameter_name',	
               'SDN:P06::id',	'SDN:P06::urn',	'SDN:P06::prefLabel',
               'sdn_uom_urn',	'sdn_uom_name']


In [41]:
# Save the result to a json dictionary make searchable by OWNER(organization) and GF3_CODE
df_dfo_output = df_dfo.set_index(['Vocabulary','name']).sort_index().applymap(str)

# Create the json file
output_dict = generate_json(df_dfo_output.replace('<NA>',''),'all_vocabulary_list.json')
df_dfo_output.replace('<NA>','').to_csv('all_vocabulary_list.csv')


In [42]:
# Show me the duplicated values (not sure why it's there)
df_dfo_output.loc[df_dfo_output.index.duplicated(keep=False)].sort_index()

Users MEDS:STATUS  ... definition convention
Vocabulary name                                            ...                      
IOS        Chlorofluorocarbon_11          IOS           A  ...                      
           Chlorofluorocarbon_11          IOS           A  ...                      
           Chlorofluorocarbon_12          IOS           A  ...                      
           Chlorofluorocarbon_12          IOS           A  ...                      
           Chlorophyll:Extracted          IOS           A  ...                      
           Chlorophyll:Extracted          IOS           A  ...                      
           Nitrogen:Particulate:Organic   IOS           A  ...                      
           Nitrogen:Particulate:Organic   IOS           A  ...                      
           Oxygen:Dissolved               IOS           A  ...                      
           Oxygen:Dissolved               IOS           A  ...                      
           Oxygen:Dissolved:SBE           IOS           A  ...                      
           Oxygen:Dissolved:SBE           IOS           A  ...                      
           Primary:Production             IOS           A  ...                      
           Primary:Production             IOS           A  ...                      
           Production:Primary             IOS           A  ...                      
           Production:Primary             IOS           A  ...                      
           Salinity                       IOS           A  ...                      
           Salinity                       IOS           A  ...                      
           Salinity                       IOS           A  ...                      
           Salinity:Bottle                IOS           A  ...                      
           Salinity:Bottle                IOS           A  ...                      
           Temperature                    IOS           A  ...                      
           Temperature                    IOS           A  ...                      
           Temperature                    IOS           A  ...                      
           Temperature                    IOS           A  ...                      
           Temperature:Primary            IOS           A  ...                      
           Temperature:Primary            IOS           A  ...                      
           Temperature:Reversing          IOS           A  ...                      
           Temperature:Reversing          IOS           A  ...                      
           Temperature:Secondary          IOS           A  ...                      
           Temperature:Secondary          IOS           A  ...                      
           Total_Suspended_Solids         IOS           A  ...                      
           Total_Suspended_Solids         IOS           A  ...                      
           Transmissivity                 IOS           A  ...                      
           Transmissivity                 IOS           A  ...                      

[35 rows x 33 columns]